In [ ]:
# import libraries
import multiprocessing as mp
import geopandas as gpd
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely import wkt
from warnings import warn
from heapq import heappop, heappush
from itertools import count

In [ ]:
# generate G of LA County
G = ox.graph_from_place('Los Angeles County, CA, USA', network_type='drive')

In [ ]:
# add attributes
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
G = ox.bearing.add_edge_bearings(G)

In [ ]:
# create gdfs
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

In [ ]:
# import O,D
origins_df = pd.read_csv("../Data/origins_gdf.csv")
destinations_df = pd.read_csv("../Data/destinations_gdf.csv")

origins_df['geometry'] = origins_df['geometry'].apply(wkt.loads)
destinations_df['geometry'] = destinations_df['geometry'].apply(wkt.loads)

# geodataframe of O, D
origins_gdf = gpd.GeoDataFrame(origins_df, geometry='geometry')
destinations_gdf = gpd.GeoDataFrame(destinations_df, geometry='geometry')

# Add attributes to G edges

Use add_edge_traffic_times function to add extra time based on traffic controls

In [ ]:
from functions import add_edge_traffic_times

In [ ]:
G = add_edge_traffic_times(G, traffic_signals_time = 30, stop_time = 15, turning_circle_time = 5, crossing_time = 5, give_way_time = 5, mini_roundabout_time = 5)

In [ ]:
# create gdfs
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)

In [ ]:
edges_gdf

In [ ]:
edges_gdf[['travel_time', 'traffic_time', 'total_time']]

# Penalty functions

Use get_turn_penalty_dict function to get turn penalty dictionary

In [ ]:
from functions import get_turn_penalty_dict

In [ ]:
penalty = get_turn_penalty_dict(G, left_turn_penalty = 30, right_turn_penalty = 10, u_turn_penalty = 90)

In [ ]:
penalty

# shortest_path_turn_penalty

In [ ]:
# import necessary functions from functions.py
from functions import shortest_path_turn_penalty
from functions import get_routes_from_gdfs

In [ ]:
# routes of 10 O-D pairs
# condiders traffic control penalty and turn penalty
routes = get_routes_from_gdfs(G, origins_gdf, destinations_gdf, weight='total_time', penalty=penalty)

In [ ]:
# travel time of 10 O-D pairs routes
time_result = []

for i in range(len(routes)):
    x = sum(ox.utils_graph.get_route_edge_attributes(G, routes[i], 'total_time'))
    time_result.append(x)

In [ ]:
time_result

In [ ]:
# format time_result
def format_travel_time(seconds):
    if seconds is None:
        return None

    rounded_seconds = round(seconds)
    minutes, remaining_seconds = divmod(rounded_seconds, 60)
    hours, minutes = divmod(minutes, 60)

    formatted_time = []

    if hours:
        formatted_time.append(f"{hours} hour")

    if minutes:
        formatted_time.append(f"{minutes} min")

    if remaining_seconds:
        formatted_time.append(f"{remaining_seconds} sec")

    return ' '.join(formatted_time)

formatted_times = [format_travel_time(t) for t in time_result]

In [ ]:
formatted_times

### TEST

In [ ]:
# sample
## for test, used 7th O-D pair
orig_node = ox.distance.nearest_nodes(G, origins_gdf.iloc[6]['geometry'].x, origins_gdf.iloc[6]['geometry'].y)
dest_node = ox.distance.nearest_nodes(G, destinations_gdf.iloc[6]['geometry'].x, destinations_gdf.iloc[6]['geometry'].y)

In [ ]:
# with both traffic, turn penalties
shortest_path = shortest_path_turn_penalty(G, orig_node, dest_node, weight="total_time", penalty=penalty)

In [ ]:
# without traffic or turn penalties
# weight = 'travel_time'
no_penalty = shortest_path_turn_penalty(G, orig_node, dest_node, weight='travel_time')

In [ ]:
# only traffic penalties
only_traffic = shortest_path_turn_penalty(G, orig_node, dest_node, weight='total_time')

In [ ]:
# only turn penalties
only_turn = shortest_path_turn_penalty(G, orig_node, dest_node, weight='travel_time', penalty=penalty)

In [ ]:
# difference in the routes
# plot the routes on top of the network
ox.plot_graph_routes(G, routes = [shortest_path, no_penalty, only_traffic, only_turn], route_linewidth=1, route_colors = ['r', 'b', 'g', 'y'], bgcolor='k', node_size=0)

plt.show()


# Calculate time

In [ ]:
# calculate the total travel time of each route
no_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, no_penalty, 'travel_time'))
all_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, shortest_path, 'total_time'))
traffic_penalties = sum(ox.utils_graph.get_route_edge_attributes(G, only_traffic, 'total_time'))
turn_penalites = sum(ox.utils_graph.get_route_edge_attributes(G, only_turn, 'travel_time'))

In [ ]:
osmnx = ox.distance.shortest_path(G, orig_node, dest_node, weight='travel_time')
osmnx_result = sum(ox.utils_graph.get_route_edge_attributes(G,osmnx, 'travel_time'))

In [ ]:
data = {
    'osmnx_result': [osmnx_result],
    'no_penalties': [no_penalties],
    'all_penalties': [all_penalties],
    'traffic_penalties': [traffic_penalties],
    'turn_penalites': [turn_penalites]
}

comparison = pd.DataFrame(data)

In [ ]:
comparison